In [78]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [11]:
# load all world bank data
# columns: 
# Country Name -> Location, 
# Country Code -> SpatialDimValueCode , 
# Year, (ww_value), ExtraFeatures
df_arg = pd.read_csv("Agriculture/agr_land.csv")
df_cpi = pd.read_csv("CPI/ConsumerPriceIndex(2010=100).csv")
df_edu = pd.read_csv("edu/school_enrollment.csv")
df_employ = pd.read_csv("Employment/EmploymentRate.csv")
df_exchange = pd.read_csv("exchange_rate/exchange_to_usd.csv")
df_GDP = pd.read_csv("GDP/GDP_per_capita.csv")
df_green = pd.read_csv("greenhouse/gas_emission.csv")
df_income = pd.read_csv("income/income_per_capita.csv")
df_pop = pd.read_csv("population/pop.csv")
df_tax = pd.read_csv("tax/tax_in_gdp.csv")

In [34]:
# unique countries
countries = df_arg['Country Name'].unique()
num_countries = len(countries)
print(num_countries)
# unique years
print(df_arg.columns) # 1960-2020 => 61
yrs = 61

266
Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020'],
      dtype='object')


In [49]:
def one_wordbank_feature(df,country):
    return df.loc[df['Country Name'] == country, '1960':'2020'].values

In [50]:
# directly create dictionary
d = dict()
d["Location"] = np.repeat(list(df_arg['Country Name']),yrs)
d["SpatialDimValueCode"] = np.repeat(list(df_arg['Country Code']),yrs)
d["Year"] = np.tile(list(df_arg.columns)[4:],num_countries)
d["arg"] = np.concatenate([one_wordbank_feature(df_arg,c) for c in countries]).ravel()
d["cpi"] = np.concatenate([one_wordbank_feature(df_cpi,c) for c in countries]).ravel()
d["edu"] = np.concatenate([one_wordbank_feature(df_edu,c) for c in countries]).ravel()
d["employ"] = np.concatenate([one_wordbank_feature(df_employ,c) for c in countries]).ravel()
d["exchange"] = np.concatenate([one_wordbank_feature(df_exchange,c) for c in countries]).ravel()
d["gdp"] = np.concatenate([one_wordbank_feature(df_GDP,c) for c in countries]).ravel()
d["green"] = np.concatenate([one_wordbank_feature(df_green,c) for c in countries]).ravel()
d["income"] = np.concatenate([one_wordbank_feature(df_income,c) for c in countries]).ravel()
d["pop"] = np.concatenate([one_wordbank_feature(df_pop,c) for c in countries]).ravel()
d["tax"] =  np.concatenate([one_wordbank_feature(df_tax,c) for c in countries]).ravel()

In [52]:
print(d)
assert(len(d["Location"]) == len(d["Year"]))
assert(len(d["arg"]) == len(d["Year"]))

{'Location': array(['Aruba', 'Aruba', 'Aruba', ..., 'Zimbabwe', 'Zimbabwe', 'Zimbabwe'],
      dtype='<U52'), 'SpatialDimValueCode': array(['ABW', 'ABW', 'ABW', ..., 'ZWE', 'ZWE', 'ZWE'], dtype='<U3'), 'Year': array(['1960', '1961', '1962', ..., '2018', '2019', '2020'], dtype='<U4'), 'arg': array([        nan, 11.11111111, 11.11111111, ..., 41.87669639,
               nan,         nan]), 'cpi': array([        nan,         nan,         nan, ..., 116.7122106,
               nan,         nan]), 'edu': array([nan, nan, nan, ..., nan, nan, nan]), 'employ': array([  nan,   nan,   nan, ...,   nan, 35.66,   nan]), 'exchange': array([        nan,         nan,         nan, ...,         nan,
               nan, 51.32901312]), 'gdp': array([        nan,         nan,         nan, ..., 1352.162653,
       1156.154864, 1128.210711]), 'green': array([   nan,    nan,    nan, ..., 31380.,    nan,    nan]), 'income': array([        nan,         nan,         nan, ..., 1394.088773,
       1208.27987 ,     

In [54]:
df_wb = pd.DataFrame(d)
df_wb.to_csv("wb.csv",index=False)

In [89]:
def happy_feature(countries, years, features):
    res = []
    for country in tqdm(countries):
        for year in years:
            try:
                entry = df_happy.loc[((df_happy['Country name'] == country) 
                                      & (df_happy['year'] == int(year))),
                                     features].values[0]
                res.append(entry)
            except:
                entry = np.nan
                res.append(entry)
    return res
    
# merge happiness table
df_happy = pd.read_csv('Happiness/WHR2021.csv')
count_hap = df_happy['Country name'].unique()
count_hap_num = len(count_hap)
dh = dict()
dh['Location'] = np.repeat(count_hap,yrs)
dh['Year'] = np.tile(list(df_arg.columns)[4:],count_hap_num)
dh['LifeLadder'] = happy_feature(count_hap,list(df_arg.columns)[4:],'Life Ladder')
dh['LogGDPPerCapita'] = happy_feature(count_hap,list(df_arg.columns)[4:],'Log GDP per capita')
dh['SocialSupport'] = happy_feature(count_hap,list(df_arg.columns)[4:],'Social support')
dh['Expectancy'] = happy_feature(count_hap,list(df_arg.columns)[4:],'Healthy life expectancy at birth')
dh['Freedom'] = happy_feature(count_hap,list(df_arg.columns)[4:],'Freedom to make life choices')
dh['Generosity'] = happy_feature(count_hap,list(df_arg.columns)[4:],'Generosity')
dh['Corruption'] = happy_feature(count_hap,list(df_arg.columns)[4:],'Perceptions of corruption')
dh['PositiveAffect'] = happy_feature(count_hap,list(df_arg.columns)[4:],'Positive affect')
dh['NegativeAffect'] = happy_feature(count_hap,list(df_arg.columns)[4:],'Negative affect')

100%|██████████| 166/166 [00:11<00:00, 14.93it/s]


In [88]:
print(dh)
assert(len(dh["LifeLadder"]) == len(dh["Year"]))

{'Location': array(['Afghanistan', 'Afghanistan', 'Afghanistan', ..., 'Zimbabwe',
       'Zimbabwe', 'Zimbabwe'], dtype=object), 'Year': array(['1960', '1961', '1962', ..., '2018', '2019', '2020'], dtype='<U4'), 'LifeLadder': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 3.7239999999999998, 4.402, 4.758, 3.832, 3.783, 3.572, 3.1310000000000002, 3.983, 4.22, 2.662, 2.694, 2.375, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 4.6339999999999995, nan, 5.485, 5.269, 5.867000000000001, 5.51, 4.551, 4.814, 4.607, 4.511, 4.64, 5.004, 4.995, 5.365, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 

In [105]:
dff_happy = pd.DataFrame(dh)
df_wbb = df_wb.merge(dff_happy, how='left')
df_wbb.to_csv("extra.csv",index=False)

In [106]:
df_wbb.head()

,Location,SpatialDimValueCode,Year,arg,cpi,edu,employ,exchange,gdp,green,...,tax,LifeLadder,LogGDPPerCapita,SocialSupport,Expectancy,Freedom,Generosity,Corruption,PositiveAffect,NegativeAffect
0,Aruba,ABW,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aruba,ABW,1961,11.111111,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aruba,ABW,1962,11.111111,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aruba,ABW,1963,11.111111,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aruba,ABW,1964,11.111111,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
df_ww = pd.read_csv("ProvidedDatasets/tobacco_use_ww.csv")
# group by Location  
df_wbb['Year'] = df_wbb['Year'].astype(str)
df_ww['Year'] = df_ww['Year'].astype(str)
df_www = df_ww.merge(df_wbb,how='left')
df_www.to_csv("extra_ww.csv",index=False)